In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import sys, os
import math
import seaborn as sns
import matplotlib.pyplot as plt

cd = os.path.split(os.getcwd())[0]
if cd not in sys.path:
    sys.path.append(cd)

from lib import noaa, bexarcrime
%matplotlib inline

[sauce A](https://www.ers.usda.gov/data-products/county-level-data-sets/county-level-data-sets-download-data/)

[sauce B](http://www.icpsr.umich.edu/icpsrweb/NACJD/studies/35019)

In [ ]:
# using crime reports, not arrests 
crime = pd.read_csv('../data/CountyCrimeReports.tsv', sep='\t')
crime['FIPS'] = crime['FIPS_ST'] * 1000 + crime['FIPS_CTY']
crime['vcrime'] = crime['MURDER'] + crime['RAPE'] + crime['ROBBERY'] + crime['AGASSLT']
crime = crime.set_index('FIPS')
crime = crime[['COVIND', 'vcrime']]

In [ ]:
edu = pd.read_excel('../data/Education.xls', skiprows=4)

# state and areas are named nicely in this dataset and will be kept for the later 'join'
# columns[-4:] include most recent data for adults eduction
# I chose the most recent because its not like the total number of HS dropouts is going to change THAT much
edu = edu[['FIPS Code', 'State', 'Area name'] + list(edu.columns[-4:])]
edu.rename(columns={'FIPS Code':'FIPS', \
                    'Area name':'County',\
                    'Percent of adults with less than a high school diploma, 2011-2015':'p_no_HS_dip', \
                    'Percent of adults with a high school diploma only, 2011-2015':'p_HS_dip',\
                    'Percent of adults completing some college or associate\'s degree, 2011-2015':'p_some_college',\
                    'Percent of adults with a bachelor\'s degree or higher, 2011-2015':'p_college_dip'}, inplace=True)
edu = edu.set_index('FIPS')

In [ ]:
pop = pd.read_excel('../data/PopulationEstimates.xls', skiprows=2)

# average the columns
cols = ['POP_ESTIMATE_2010','POP_ESTIMATE_2011','POP_ESTIMATE_2012','POP_ESTIMATE_2013','POP_ESTIMATE_2014','POP_ESTIMATE_2015','POP_ESTIMATE_2016']
pop['avgpop'] = pop[cols].sum(axis=1) / len(cols)

# more averaging
cols = ['N_POP_CHG_2010','N_POP_CHG_2011','N_POP_CHG_2012','N_POP_CHG_2013','N_POP_CHG_2014','N_POP_CHG_2015','N_POP_CHG_2016']
pop['dpop/dt'] = pop[cols].sum(axis=1) / len(cols)

# only pull FIPS code, population, and dp
pop = pop[['FIPS', 'avgpop', 'dpop/dt']]
pop = pop.set_index('FIPS')

In [ ]:
pov = pd.read_excel('../data/PovertyEstimates.xls', skiprows=3)
# only select poverty percentage
pov = pov[['FIPStxt', 'PCTPOVALL_2015']]
pov.rename(columns={'FIPStxt':'FIPS', 'PCTPOVALL_2015':'p_impoverished'}, inplace=True)
pov = pov.set_index('FIPS')
pov.p_impoverished = pd.to_numeric(pov.p_impoverished, errors='coerce')

In [ ]:
emp = pd.read_excel('../data/Unemployment.xls', skiprows=9)

#avg unemployment
cols = ['Unemployment_rate_2007', 'Unemployment_rate_2008', 'Unemployment_rate_2009', 'Unemployment_rate_2010', 'Unemployment_rate_2011', 'Unemployment_rate_2012', 'Unemployment_rate_2013', 'Unemployment_rate_2014', 'Unemployment_rate_2015', 'Unemployment_rate_2016']
emp['p_unempl'] = emp[cols].sum(axis=1) / len(cols)

#only pull average and income
emp = emp[['FIPStxt', 'p_unempl', 'Median_Household_Income_2015']]
emp.rename(columns={'FIPStxt':'FIPS', 'Median_Household_Income_2015':'med_income'}, inplace=True)
emp = emp.set_index('FIPS')

In [ ]:
df = edu.join([pop,pov,emp,crime], how='outer')
df = df.where(df.State != 'PR').dropna(how='all') ## Puerto Rico has unreliable data

#pull out nationwide data
us = df.iloc[0]
df = df.drop(0)

In [ ]:
#pull out statewide data
s = [x for x in range(1000,75000,1000)]
states = df.loc[s].dropna(how='all')

# all thats left is county level data
df = df.drop(states.index)

In [ ]:
#normalizing data
df['p_dpop'] = df['dpop/dt']/df['avgpop']
df['vcrime_rate'] = 100000 * df['vcrime']/df['avgpop']
df = df.drop(['dpop/dt', 'vcrime'], axis=1)

In [ ]:
df.corr()

In [ ]:
df.describe()

In [ ]:
# According to the 2008 presidential election
blue_states =['WA', 'OR', 'CA', 'NV', 'NM', 'CO', 'MN', 'IA', 'WI', 'IL', 'IN', 'MI', 'OH', 'PA', 'NY', 'VT', 'NH', 'ME', 'MA', 'CT', 'RI', 'NJ', 'DE', 'MD', 'VA', 'NC', 'FL', 'HI']
red_states = ['ID', 'MT', 'WY', 'UT', 'AZ', 'ND', 'SD', 'NE', 'KS', 'OK', 'TX', 'MO', 'AR', 'LA', 'WV', 'KY', 'TN', 'MS', 'AL', 'GA', 'SC', 'AK']
fix, ax = plt.subplots(figsize=(20,10))
pal = {state: 'r' if state in red_states else "b" for state in df.State}
sns.boxplot(ax=ax, x='State', y='vcrime_rate', data=df, palette=pal)
#sns.boxplot(ax=ax2, x='State', y='vcrime_rate', data=df.where(df.State.isin(red_states)))

In [ ]:
fix, ax = plt.subplots(figsize=(20,10))
sns.boxplot(ax=ax, x='State', y='p_no_HS_dip', data=df, palette=pal)

Graphs of factors to violent crime

In [ ]:
sns.pairplot(df, y_vars=['vcrime_rate'], x_vars=['p_no_HS_dip', 'p_HS_dip', 'p_some_college', 'p_college_dip', 'avgpop',
       'p_impoverished', 'p_unempl', 'med_income', 'p_dpop', 'vcrime_rate'], dropna=True, size=10)

In [ ]:
sns.distplot(df.vcrime_rate.dropna(), axlabel="Violent crime per 100,000")

In [ ]:
sns.distplot(df.avgpop.dropna().apply(np.log10), axlabel="Population (log10)")

In [ ]:
sns.distplot(df.p_unempl.dropna(), axlabel='Unemployment Rate')

In [ ]:
sns.distplot(df.p_impoverished.dropna(), axlabel="Poverty Rate")

In [ ]:
# bins data into high, medium, and low (based on national quantiles) for grouping
binned = pd.DataFrame({c : pd.qcut(df[c], 3, labels=['L', 'M', 'H']) for c in df.drop(['State', 'County', 'COVIND'], axis=1).columns}).join(df[['State', 'County', 'COVIND']])

In [ ]:
# apparently the worst counties to be in in Texas
# High violent crime rate, high rate of unemployment, and high populations
TX = binned.dropna(how='all').groupby(['vcrime_rate', 'p_unempl', 'avgpop'])
df.loc[TX.get_group(('H', 'H', 'H')).index].where(df.State == 'TX').dropna()

In [ ]:
sns.distplot(df.where(df.State=='TX').vcrime_rate.dropna(), label="Violent Crime Rates in Texas")
sns.distplot(df.vcrime_rate.dropna(), axlabel="Violent Crime Rates in US and Texas")

In [ ]:
#highest crime counties in US
# note that high city crime does not necessarily match high county crime
# eg: chicago is high crime, but it's split between 2 counties
# St Louis has the highest crime, but it's its own county, so it tops this list as well
df.where(df.avgpop > 10000).sort_values('vcrime_rate', ascending=False)[:20]

Group the data by violent crime rate, poverty rate, unemployment rate, and population

In [ ]:
groups = ['vcrime_rate', 'p_impoverished', 'p_unempl', 'avgpop']

In [ ]:
# all counties grouped by H/M/L rates of whatever
c = binned.dropna(how='all').groupby(groups[::-1])
c.count().where(c.count().State > 10).dropna().sort_values('State', ascending=False)['State'].unstack()

We select counties with high rates of enemployment, violent crime, poverty, and large populations sampled using a nonrandom seed

In [ ]:
# get the original values of the first row ^^^
# such that cities have high crime rate, high poverty, High unemployment, High population, etc
selection = ('H','H','H', 'H')
for x in groups:
    print("%10s " %x[:10], end='')
print('')
for x in selection:
    print("%10s " %x[:10], end='')
HHHstates = df.loc[c.get_group(selection).index]
HHHstates.where(HHHstates.vcrime_rate > 800).dropna().sample(10, random_state=15)

From those counties, we selected the county seats as the cities
We looked up the latitude and longitude of them to match to NOAA's list of weather stations. We used Pythagoras' theorem to find the closest station to the city, as some cities may not have one within city limits.

In [ ]:
cities = {
    'Philidelphia, PA' : (39.9526, -75.1652), #Philadelphia County
    'Albany, GA' : (31.5785, -84.1557), # Gougherty County
    'Memphis, TN' : (35.1495, -90.0490), # Shelby County and Crittenden County
    'Toledo, OH' : (41.6639, -83.5552), # Lucas County
    'Pine Bluff AR' : (34.2284, -92.0032), # Jefferson County
    'Detroit, MI' : (42.3314, -83.0458), # Wayne County
    'Baltimore, MD' : (39.2904, -76.6122), # Baltimore City
    'Flint, MI' : (43.0125, -83.6875), # Genesee County
    'St. Louis, MO' : (38.6270, -90.1994) # St. Louis City
}

In [ ]:
# list of stations with location, name, and recording beginning and end dates
hist = pd.read_csv('ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.csv')

In [ ]:
# only recent stations
hist = hist.where(hist.END > 20120101 ).dropna(how='all')

In [ ]:
def dist(a, b):
    return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

In [ ]:
def format_station_code(usaf, wban):
    usafstr = str(int(usaf))
    wbanstr = str(int(wban))
    
    if len(usafstr) < 6:
        usafstr = '0'*(6-len(usafstr)) + usafstr
        
    if len(wbanstr) < 5:
        wbanstr = '0'*(5-len(wbanstr)) + wbanstr
        
    return usafstr + '-' + wbanstr

In [ ]:
stations = dict()
for city in cities.keys():
    coord = cities[city]
    mindist = 999
    minindex = 0
    for index, row in hist.iterrows():
        d = dist(coord, (row['LAT'], row['LON']))
        if (d < mindist):
            mindist = d
            minindex = index
    print('Nearest ({:^6.2f}) ISD to {:20} is {:40} at loc {}'.format(mindist, city, hist.loc[minindex]['STATION NAME'], minindex))
    stations[city] = format_station_code(hist.loc[minindex]['USAF'], hist.loc[minindex]['WBAN'])
    print('\tStation code is {}'.format(stations[city]))